In [6]:
#real time


import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# 모델 로드
loaded_model = tf.keras.models.load_model('keras_model.h5')

# CSV 파일에 헤더를 추가
csv_filename = 'realtime_data.csv'
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['R', 'G', 'B', 'Openness', 'Width'])  # Label은 추후 예측 시 추가할 수 있음

# Mediapipe의 Face Mesh 모델 로드
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# 웹캠에서 영상을 받아오는 VideoCapture 객체 생성
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("비디오 캡처 실패, 종료합니다.")
        break

    frame = cv2.flip(frame, 1)
    
    # 프레임을 BGR에서 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 얼굴 검출 및 랜드마크 추출
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmark_indices = [0, 61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 37, 39, 40, 267, 269, 270]
            landmarks = [face_landmarks.landmark[i] for i in landmark_indices]
            
            h, w, _ = frame.shape
            x_1 = int(np.mean([landmark.x * w for landmark in landmarks]))
            y_1 = int(np.mean([landmark.y * h for landmark in landmarks])) - 5
            
            # RGB 값 추출
            rgb_value = rgb_frame[y_1, x_1]
            r, g, b = rgb_value[0], rgb_value[1], rgb_value[2]
            
            # 얼굴 너비 계산
            face_width = math.sqrt(
                (face_landmarks.landmark[172].x - face_landmarks.landmark[264].x) ** 2 +
                (face_landmarks.landmark[172].y - face_landmarks.landmark[264].y) ** 2
            )
            face_width = 20 * face_width

            # 입의 세로길이 계산 (고중저모음 판단기준)
            upper_lip_bottom = (face_landmarks.landmark[12].x, face_landmarks.landmark[12].y)
            lower_lip_top = (face_landmarks.landmark[14].x, face_landmarks.landmark[14].y)
            mouth_openness = (lower_lip_top[1] - upper_lip_bottom[1]) / face_width
            mouth_openness = round(mouth_openness, 4)

            # 입의 가로길이 계산 (원순/평순 판단기준)
            lip_left = (face_landmarks.landmark[61].x, face_landmarks.landmark[61].y)
            lip_right = (face_landmarks.landmark[291].x, face_landmarks.landmark[291].y)
            mouth_width = (lip_right[0] - lip_left[0]) / face_width
            mouth_width = round(mouth_width, 4)

            # CSV 파일에 데이터 추가
            with open(csv_filename, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([r, g, b, mouth_openness, mouth_width])

            # 데이터 표준화 및 예측
            new_data = pd.read_csv(csv_filename)
            X = new_data[['R', 'G', 'B', 'Openness', 'Width']].values

            scaler = StandardScaler()
            scaled_data = scaler.fit_transform(X)

            # 모델을 사용하여 예측 수행
            predictions = loaded_model.predict(scaled_data)

            # 예측 결과 출력
            for i, prediction in enumerate(predictions):
                predicted_class = np.argmax(prediction)  # 가장 높은 값(확률)을 가지는 클래스 선택
                print(f"새로운 데이터 {i+1}의 예측 클래스: {predicted_class}")

    # 영상 출력
    cv2.imshow('Face Mesh', frame)
    
    # 종료 키 (q) 입력 시 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 

# 자원 해제
cap.release()
cv2.destroyAllWindows()


c:\anaconda\envs\AI\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
새로운 데이터 1의 예측 클래스: 3


c:\anaconda\envs\AI\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
새로운 데이터 1의 예측 클래스: 2
새로운 데이터 2의 예측 클래스: 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
새로운 데이터 1의 예측 클래스: 5
새로운 데이터 2의 예측 클래스: 0
새로운 데이터 3의 예측 클래스: 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

c:\anaconda\envs\AI\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\anaconda\envs\AI\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

c:\anaconda\envs\AI\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


새로운 데이터 1의 예측 클래스: 5
새로운 데이터 2의 예측 클래스: 2
새로운 데이터 3의 예측 클래스: 2
새로운 데이터 4의 예측 클래스: 2
새로운 데이터 5의 예측 클래스: 2
새로운 데이터 6의 예측 클래스: 2
새로운 데이터 7의 예측 클래스: 4
새로운 데이터 8의 예측 클래스: 4
새로운 데이터 9의 예측 클래스: 4
새로운 데이터 10의 예측 클래스: 4
새로운 데이터 11의 예측 클래스: 4
새로운 데이터 12의 예측 클래스: 4
새로운 데이터 13의 예측 클래스: 4
새로운 데이터 14의 예측 클래스: 4
새로운 데이터 15의 예측 클래스: 2
새로운 데이터 16의 예측 클래스: 5
새로운 데이터 17의 예측 클래스: 5
새로운 데이터 18의 예측 클래스: 5
새로운 데이터 19의 예측 클래스: 2
새로운 데이터 20의 예측 클래스: 5
새로운 데이터 21의 예측 클래스: 5
새로운 데이터 22의 예측 클래스: 5
새로운 데이터 23의 예측 클래스: 5
새로운 데이터 24의 예측 클래스: 5
새로운 데이터 25의 예측 클래스: 5
새로운 데이터 26의 예측 클래스: 4
새로운 데이터 27의 예측 클래스: 6
새로운 데이터 28의 예측 클래스: 6
새로운 데이터 29의 예측 클래스: 6
새로운 데이터 30의 예측 클래스: 6
새로운 데이터 31의 예측 클래스: 6
새로운 데이터 32의 예측 클래스: 6
새로운 데이터 33의 예측 클래스: 6
새로운 데이터 34의 예측 클래스: 6
새로운 데이터 35의 예측 클래스: 6
새로운 데이터 36의 예측 클래스: 6
새로운 데이터 37의 예측 클래스: 6
새로운 데이터 38의 예측 클래스: 6
새로운 데이터 39의 예측 클래스: 6
새로운 데이터 40의 예측 클래스: 6
새로운 데이터 41의 예측 클래스: 6
새로운 데이터 42의 예측 클래스: 6
새로운 데이터 43의 예측 클래스: 6
새로운 데이터 44의 예측 클래스: 6
새로운 데이터 45의 예측 클래스: 6
새로운 데이터 46의 예측 클래스:

c:\anaconda\envs\AI\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


IndexError: index 492 is out of bounds for axis 0 with size 480

: 